In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, roc_auc_score
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import lightgbm as lgb
import pandas_datareader.data as web


lookback_days = 120

n_splits = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [2]:
dis_df = yf.download("DIS", start="2010-01-01", end="2025-05-31", auto_adjust=True)
dis_df.columns = dis_df.columns.get_level_values(0)

sp500_df = yf.download("^GSPC", start="2010-01-01", end="2025-05-31", auto_adjust=True)
sp500_df.columns = sp500_df.columns.get_level_values(0)

vix_df = yf.download("^VIX", start="2010-01-01", end="2025-05-31", auto_adjust=True)
vix_df.columns = vix_df.columns.get_level_values(0)

data = dis_df[["Open", "High", "Low", "Close", "Volume"]].copy()
data.rename(columns={"Close": "Adj Close"}, inplace=True)

sp500_adj = sp500_df["Close"].rename("SP500")
vix_adj   = vix_df["Close"].rename("VIX")

data = data.join(sp500_adj, how="inner").join(vix_adj, how="inner")
data.dropna(inplace=True)

data.tail()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Adj Close,Volume,SP500,VIX
Date,,,,,,,
2025-05-23,109.599998,110.349998,108.779999,109.720001,8086100,5802.819824,22.290001
2025-05-27,111.029999,112.889999,110.150002,112.360001,10201100,5921.540039,18.959999
2025-05-28,112.199997,112.470001,111.169998,111.519997,5787900,5888.549805,19.309999
2025-05-29,112.010002,112.260002,110.519997,112.019997,9682100,5912.169922,19.180000
2025-05-30,112.010002,113.349998,111.370003,113.040001,12878800,5911.689941,18.570000


In [3]:
yields = web.DataReader(["DGS10", "DGS2"], data_source="fred", start="2010-01-01", end="2025-05-31")

yields.ffill(inplace=True)

yields["term_spread"] = yields["DGS10"] - yields["DGS2"]
yields["term_spread_chg_1d"] = yields["term_spread"].diff(periods=1)
yields["term_spread_chg_3d"] = yields["term_spread"].diff(periods=3)
yields.dropna(subset=["term_spread_chg_3d"], inplace=True)


In [4]:
ppi = web.DataReader("PPIACO", data_source="fred", start="2010-01-01", end="2025-05-31")

ppi = ppi.ffill()

ppi_daily = ppi.resample("D").ffill()

ppi_daily["ppi_chg_1m"] = ppi_daily["PPIACO"].pct_change(periods=30)

ppi_daily["ppi_chg_3m"] = ppi_daily["PPIACO"].pct_change(periods=90)

ppi_daily.dropna(subset=["ppi_chg_3m"], inplace=True)


In [5]:
xly_df = yf.download("XLY", start="2010-01-01", end="2025-05-30", auto_adjust=True)
xly_df.columns = xly_df.columns.get_level_values(0)

# 3) Compute 1-day and 3-day log-returns for XLY
xly_df["XLY_ret_1d"] = np.log(xly_df["Close"] / xly_df["Close"].shift(1))
xly_df["XLY_ret_3d"] = np.log(xly_df["Close"] / xly_df["Close"].shift(3))

# 4) Compute 50-day and 200-day simple moving averages, plus the ratio to 200-day MA
xly_df["XLY_SMA_50"]  = xly_df["Close"].rolling(window=50,  min_periods=50).mean()
xly_df["XLY_SMA_200"] = xly_df["Close"].rolling(window=200, min_periods=200).mean()

# To capture the “cross‐below‐200‐day‐MA” effect, you can use the ratio:
xly_df["XLY_SMA_ratio"] = xly_df["Close"] / xly_df["XLY_SMA_200"]

# 5) You only need these new columns, so drop the rest of XLY’s columns before merging:
xly_feats = xly_df[["XLY_ret_1d", "XLY_ret_3d", "XLY_SMA_50", "XLY_SMA_200", "XLY_SMA_ratio"]].copy()


[*********************100%***********************]  1 of 1 completed


In [6]:
def compute_technical_indicators(df):
    df = df.copy()
    df["log_ret"] = np.log(df["Adj Close"] / df["Adj Close"].shift(1))
    df["SMA_5"]   = df["Adj Close"].rolling(5).mean()
    df["SMA_10"]  = df["Adj Close"].rolling(10).mean()
    df["SMA_20"]  = df["Adj Close"].rolling(20).mean()
    df["EMA_10"]  = df["Adj Close"].ewm(span=10, adjust=False).mean()
    df["EMA_20"]  = df["Adj Close"].ewm(span=20, adjust=False).mean()
    df["MOM_10"]  = df["Adj Close"] / df["Adj Close"].shift(10) - 1

    ema12 = df["Adj Close"].ewm(span=12, adjust=False).mean()
    ema26 = df["Adj Close"].ewm(span=26, adjust=False).mean()
    df["MACD"]    = ema12 - ema26
    df["MACD_SIG"] = df["MACD"].ewm(span=9, adjust=False).mean()

    rolling20 = df["Adj Close"].rolling(20)
    df["BB_MID"]   = rolling20.mean()
    df["BB_UP"]    = rolling20.mean() + 2 * rolling20.std()
    df["BB_LOW"]   = rolling20.mean() - 2 * rolling20.std()
    df["BB_WIDTH"]= (df["BB_UP"] - df["BB_LOW"]) / df["BB_MID"]

    delta = df["Adj Close"].diff()
    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)
    roll_up = up.rolling(14).mean()
    roll_down = down.rolling(14).mean()
    rs = roll_up / roll_down
    df["RSI_14"] = 100 - (100 / (1 + rs))

    high_low         = df["High"] - df["Low"]
    high_close_prev  = (df["High"] - df["Adj Close"].shift(1)).abs()
    low_close_prev   = (df["Low"] - df["Adj Close"].shift(1)).abs()
    tr = pd.concat([high_low, high_close_prev, low_close_prev], axis=1).max(axis=1)
    df["ATR_14"] = tr.rolling(14).mean()

    direction = np.sign(df["Adj Close"].diff()).fillna(0)
    df["OBV"] = (direction * df["Volume"]).cumsum()

    rolling_ret = df["log_ret"].rolling(10)
    df["Vol_10"]  = rolling_ret.std()
    df["Skew_10"] = rolling_ret.apply(lambda x: x.skew(), raw=False)
    df["Kurt_10"] = rolling_ret.apply(lambda x: x.kurt(), raw=False)

    pv = df["Adj Close"] * df["Volume"]
    df["VWAP_14"] = pv.rolling(14).sum() / df["Volume"].rolling(14).sum()

    df["SP500_ret"] = np.log(df["SP500"] / df["SP500"].shift(1))
    df["VIX_ret"]   = np.log(df["VIX"] / df["VIX"].shift(1))

    # volume indicators
    vix_75 = df["VIX"].rolling(window=60, min_periods=60).quantile(0.75)
    vix_90 = df["VIX"].rolling(window=60, min_periods=60).quantile(0.90)

    # 2. Define vol_state: 0 = normal (<=75%), 1 = elevated (75–90), 2 = crisis (>90)
    df["vol_state"] = [
        2 if v > q90 else (1 if v > q75 else 0)
        for v, q75, q90 in zip(df["VIX"], vix_75, vix_90)
    ]

    df["vol_state"] = df["vol_state"].astype(int)
    df["vol_norm"]   = (df["vol_state"] == 0).astype(int)  # normal volatility
    df["vol_elev"]   = (df["vol_state"] == 1).astype(int)  # elevated volatility
    df["vol_crisis"] = (df["vol_state"] == 2).astype(int)  # crisis volatility

    # Add date-based features
    df['Day_of_Week'] = df.index.dayofweek  # 0=Monday, 6=Sunday
    df['Month_of_Year'] = df.index.month  # 1=Jan, 12=Dec

    # Convert to cyclical features
    df['DoW_sin'] = np.sin(2 * np.pi * df['Day_of_Week'] / 7)
    df['DoW_cos'] = np.cos(2 * np.pi * df['Day_of_Week'] / 7)
    df['MoY_sin'] = np.sin(2 * np.pi * (df['Month_of_Year'] - 1) / 12)
    df['MoY_cos'] = np.cos(2 * np.pi * (df['Month_of_Year'] - 1) / 12)

    # Add term spread features
    df = df.join(yields[["term_spread", "term_spread_chg_1d", "term_spread_chg_3d"]], how="inner")

    # Add PPI features
    df = df.join(ppi_daily[["ppi_chg_1m", "ppi_chg_3m"]], how="inner")

    # Add XLY features
    # df = df.join(xly_feats, how="inner")

    return df

In [7]:
features = compute_technical_indicators(data)
features.dropna(inplace=True)
features["target"] = features["log_ret"].shift(-1)
features.dropna(inplace=True)

feature_cols = [
    "SMA_5", "SMA_10", "SMA_20",
    "EMA_10", "EMA_20",
    "MOM_10",
    "MACD", "MACD_SIG",
    "BB_MID", "BB_UP", "BB_LOW", "BB_WIDTH",
    "RSI_14",
    "ATR_14",
    "OBV",
    "Vol_10", "Skew_10", "Kurt_10",
    "VWAP_14",
    "SP500_ret", "VIX_ret",
    "vol_norm", "vol_elev", "vol_crisis",
    "Day_of_Week", "Month_of_Year",
    "DoW_sin", "DoW_cos", "MoY_sin", "MoY_cos",
    "term_spread", "term_spread_chg_1d", "term_spread_chg_3d",
    "ppi_chg_1m", "ppi_chg_3m",
    # "XLY_ret_1d", "XLY_ret_3d", "XLY_SMA_50", "XLY_SMA_200", "XLY_SMA_ratio"
]

features.tail(n=20)

,Open,High,Low,Adj Close,Volume,SP500,VIX,log_ret,SMA_5,SMA_10,...,DoW_sin,DoW_cos,MoY_sin,MoY_cos,term_spread,term_spread_chg_1d,term_spread_chg_3d,ppi_chg_1m,ppi_chg_3m,target
2025-03-04,111.699997,112.250000,108.269997,109.010002,12658000,5778.149902,23.510000,-0.034088,111.748001,111.103001,...,0.781831,0.623490,0.866025,0.5,0.26,6.000000e-02,0.04,-0.003352,0.020866,0.002748
2025-03-05,109.339996,110.940002,108.220001,109.309998,9956100,5842.629883,21.930000,0.002748,111.244000,110.899001,...,0.974928,-0.222521,0.866025,0.5,0.29,3.000000e-02,0.04,-0.003352,0.020866,-0.036141
2025-03-06,108.379997,108.830002,105.019997,105.430000,11618800,5738.520020,24.870001,-0.036141,110.068001,110.387000,...,0.433884,-0.900969,0.866025,0.5,0.33,4.000000e-02,0.13,-0.003352,0.020866,0.000759
2025-03-07,105.000000,105.769997,103.309998,105.510002,12715900,5770.200195,23.370001,0.000759,108.410001,110.072000,...,-0.433884,-0.900969,0.866025,0.5,0.33,0.000000e+00,0.07,-0.003352,0.020866,-0.023300
2025-03-10,104.430000,104.800003,102.110001,103.080002,10947100,5614.560059,27.860001,-0.023300,106.468001,109.260001,...,0.000000,1.000000,0.866025,0.5,0.33,-4.440892e-16,0.04,-0.003352,0.020866,-0.051559
2025-03-11,99.790001,101.040001,97.449997,97.900002,16906000,5572.069824,26.920000,-0.051559,104.246001,107.997001,...,0.781831,0.623490,0.866025,0.5,0.34,1.000000e-02,0.01,-0.003352,0.020866,0.008746
2025-03-12,98.680000,99.489998,96.379997,98.760002,11891800,5599.299805,24.230000,0.008746,102.136002,106.690001,...,0.974928,-0.222521,0.866025,0.5,0.31,-3.000000e-02,-0.02,-0.003352,0.020866,-0.019323
2025-03-13,97.769997,98.650002,95.930000,96.870003,11059500,5521.520020,24.660000,-0.019323,100.424002,105.246001,...,0.433884,-0.900969,0.866025,0.5,0.33,2.000000e-02,0.00,-0.003352,0.020866,0.018107
2025-03-14,97.440002,99.099998,97.419998,98.639999,10199800,5638.939941,21.770000,0.018107,99.050002,103.730001,...,-0.433884,-0.900969,0.866025,0.5,0.29,-4.000000e-02,-0.05,-0.003352,0.020866,0.007273
2025-03-17,98.739998,100.120003,98.660004,99.360001,8332100,5675.120117,20.510000,0.007273,98.306001,102.387001,...,0.000000,1.000000,0.866025,0.5,0.25,-4.000000e-02,-0.06,-0.003352,0.020866,-0.000101


In [8]:
X_full = features[feature_cols].values
y_full = features["target"].values

def create_sequences(X_np, y_np, lookback):
    Xs, ys = [], []
    for i in range(lookback, len(X_np)):
        Xs.append(X_np[i - lookback:i, :])
        ys.append(y_np[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_full, y_full, lookback_days)
print("Sequence shapes → X_seq:", X_seq.shape, "y_seq:", y_seq.shape)

Sequence shapes → X_seq: (3653, 120, 35) y_seq: (3653,)


In [9]:

tscv = TimeSeriesSplit(n_splits=n_splits)
fold_metrics = []

for fold, (train_idx, val_idx) in enumerate(tscv.split(X_seq), start=1):
    print(f"--- Fold {fold}/{n_splits} ---")
    X_train_seq, y_train_seq = X_seq[train_idx], y_seq[train_idx]
    X_val_seq,   y_val_seq   = X_seq[val_idx],   y_seq[val_idx]

    # Scale by flattening
    F = X_full.shape[1]
    X_train_flat = X_train_seq.reshape(-1, F)
    scaler = StandardScaler().fit(X_train_flat)

    def scale_sequences(Xs):
        n_s, seq_len, n_f = Xs.shape
        Xf = Xs.reshape(-1, n_f)
        Xf_sc = scaler.transform(Xf)
        return Xf_sc.reshape(n_s, seq_len, n_f)

    X_train_seq_sc = scale_sequences(X_train_seq)
    X_val_seq_sc   = scale_sequences(X_val_seq)

    train_dataset = TensorDataset(
        torch.tensor(X_train_seq_sc, dtype=torch.float32),
        torch.tensor(y_train_seq,  dtype=torch.float32).unsqueeze(1)
    )
    val_dataset   = TensorDataset(
        torch.tensor(X_val_seq_sc, dtype=torch.float32),
        torch.tensor(y_val_seq,   dtype=torch.float32).unsqueeze(1)
    )

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False, drop_last=True)
    val_loader   = DataLoader(val_dataset,   batch_size=32, shuffle=False, drop_last=False)

    class BayesianLSTM(nn.Module):
        def __init__(self, input_size, hidden_size=128, num_layers=3, dropout=0.3):
            super().__init__()
            # A bidirectional LSTM: output dim is hidden_size * 2
            self.lstm = nn.LSTM(
                input_size=input_size,
                hidden_size=hidden_size,
                num_layers=num_layers,
                batch_first=True,
                dropout=dropout,
                bidirectional=True
            )
            # Attention projection from (hidden_size*2) → 1
            self.attn_proj = nn.Linear(hidden_size * 2, 1)
            # Dropout that you can leave “on” at inference for MC‐Dropout
            self.dropout = nn.Dropout(dropout)
            # Final regression from (hidden_size*2) → 1
            self.fc = nn.Linear(hidden_size * 2, 1)

        def forward(self, x):
            # x: (batch, seq_len, input_size)
            out, _ = self.lstm(x)
            # out: (batch, seq_len, hidden_size*2)

            # Compute attention scores (unnormalized) for each time step
            # attn_scores: (batch, seq_len, 1)
            attn_scores = self.attn_proj(out)

            # Normalize over the seq_len dimension to get attention weights
            # attn_weights: (batch, seq_len, 1)
            attn_weights = torch.softmax(attn_scores, dim=1)

            # Compute context vector as weighted sum of LSTM outputs
            # context: (batch, hidden_size*2)
            context = torch.sum(attn_weights * out, dim=1)

            # Apply dropout to context (this is your MC‐Dropout step)
            context = self.dropout(context)

            # Final regression to one scalar
            return self.fc(context)


    model = BayesianLSTM(input_size=F).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", factor=0.5, patience=5,
    )

    def directional_hinge(r_pred, r_true):
        return torch.relu(-r_pred * r_true).mean()

    alpha = 0.3

    best_val_loss = np.inf
    patience_cnt = 0
    max_epochs = 50
    early_stop_patience = 5

    for epoch in range(1, max_epochs + 1):
        model.train()
        train_losses = []

        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)

            optimizer.zero_grad()
            r_pred   = model(xb)                     # shape (batch, 1)
            loss_mse = criterion(r_pred, yb)
            loss_dir = directional_hinge(r_pred, yb)  # positive if sign(r_pred) != sign(r_true)

            loss = loss_mse + alpha * loss_dir
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())


        model.eval()
        val_losses = []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                val_losses.append(criterion(model(xb), yb).item())

        avg_val_loss = np.mean(val_losses)
        scheduler.step(avg_val_loss)

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), f"lstm_fold{fold}_best.pth")
            patience_cnt = 0
        else:
            patience_cnt += 1
            if patience_cnt >= early_stop_patience:
                break

    model.load_state_dict(torch.load(f"lstm_fold{fold}_best.pth"))
    model.eval()

    preds_val = []
    with torch.no_grad():
        for xb, _ in val_loader:
            xb = xb.to(device)
            preds_val.append(model(xb).cpu().numpy().flatten())

    preds_val = np.concatenate(preds_val)
    rmse_val = np.sqrt(mean_squared_error(y_val_seq, preds_val))
    mae_val  = mean_absolute_error(y_val_seq, preds_val)
    hit_val  = np.mean(np.sign(preds_val) == np.sign(y_val_seq))

    print(f"Fold {fold} → RMSE: {rmse_val:.5f}, MAE: {mae_val:.5f}, Hit Ratio: {hit_val:.3f}")
    fold_metrics.append((rmse_val, mae_val, hit_val))

rmse_list, mae_list, hit_list = zip(*fold_metrics)
print("CV RMSE:       {:.5f} ± {:.5f}".format(np.mean(rmse_list), np.std(rmse_list)))
print("CV MAE:        {:.5f} ± {:.5f}".format(np.mean(mae_list), np.std(mae_list)))
print("CV Hit Ratio:  {:.3f} ± {:.3f}".format(np.mean(hit_list), np.std(hit_list)))


--- Fold 1/5 ---
Fold 1 → RMSE: 0.01115, MAE: 0.00825, Hit Ratio: 0.564
--- Fold 2/5 ---
Fold 2 → RMSE: 0.01196, MAE: 0.00815, Hit Ratio: 0.492
--- Fold 3/5 ---
Fold 3 → RMSE: 0.02028, MAE: 0.01256, Hit Ratio: 0.464
--- Fold 4/5 ---
Fold 4 → RMSE: 0.01977, MAE: 0.01421, Hit Ratio: 0.526
--- Fold 5/5 ---
Fold 5 → RMSE: 0.01783, MAE: 0.01210, Hit Ratio: 0.487
CV RMSE:       0.01620 ± 0.00389
CV MAE:        0.01105 ± 0.00243
CV Hit Ratio:  0.507 ± 0.035


In [10]:
# Cell 5: Retrain on All Data and One-Day-Ahead Forecast

# Scale full data
X_full_flat = X_seq.reshape(-1, X_full.shape[1])
scaler_full = StandardScaler().fit(X_full_flat)
X_seq_sc = scaler_full.transform(X_full_flat).reshape(X_seq.shape)

dataset_full = TensorDataset(
    torch.tensor(X_seq_sc, dtype=torch.float32),
    torch.tensor(y_seq,   dtype=torch.float32).unsqueeze(1)
)
loader_full = DataLoader(dataset_full, batch_size=32, shuffle=False, drop_last=True)

model_full = BayesianLSTM(input_size=F).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_full.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=5
)

best_loss = np.inf
patience_cnt = 0
max_epochs = 100
early_stop_patience = 10

for epoch in range(1, max_epochs + 1):
    model_full.train()
    train_losses = []
    for xb, yb in loader_full:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model_full(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    avg_train_loss = np.mean(train_losses)
    scheduler.step(avg_train_loss)

    if avg_train_loss < best_loss:
        best_loss = avg_train_loss
        torch.save(model_full.state_dict(), "lstm_full_best.pth")
        patience_cnt = 0
    else:
        patience_cnt += 1
        if patience_cnt >= early_stop_patience:
            break

model_full.load_state_dict(torch.load("lstm_full_best.pth"))
model_full.eval()

# One-Day-Ahead Forecast
last_seq = X_full[-lookback_days:].copy()
last_seq_sc = scaler_full.transform(last_seq)
X_new = last_seq_sc.reshape(1, lookback_days, X_full.shape[1])

with torch.no_grad():
    xb_tensor = torch.tensor(X_new, dtype=torch.float32).to(device)
    pred_log_ret = model_full(xb_tensor).cpu().numpy().flatten()[0]

last_price = features["Adj Close"].iloc[-1]
predicted_price = last_price * np.exp(pred_log_ret)

print(f"Predicted DIS log-ret (t+1): {pred_log_ret:.6f}")
print(f"Yesterday’s Adj Close:     {last_price:.4f}")
print(f"Forecasted Adj Close (t+1): {predicted_price:.4f}")


Predicted DIS log-ret (t+1): 0.002673
Yesterday’s Adj Close:     98.7000
Forecasted Adj Close (t+1): 98.9641
